In [1]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import pymongo
import time

#chromedriver set up
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [2]:
#Visit url for news data
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [3]:
#convert parse html with Beautiful Soup, scrape first title and paragraph
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

news_title = soup.find('div',class_='content_title').text.strip()
news_p = soup.find('div',class_='article_teaser_body').text.strip()

In [4]:
#Visit url for Mars image
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [5]:
#create soup instance and scrape image url
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

article = soup.find('article',class_='carousel_item')
img_url1 = 'https://www.jpl.nasa.gov'
img_url2 = article['style'].split("'")[1]

featured_image_url = img_url1 + img_url2

In [6]:
#Visit url for Mars Weather Twitter
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [7]:
#create soup instance and scrape latest weather
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

mars_weather = soup.find('p',class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text

In [8]:
#Visit url for Mars Facts
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [9]:
#parse url to retrieve Mars table and set column headers
tables = pd.read_html(url)
df = tables[0]
df.columns = ['Mars Fact', 'Value']
df.head()

,Mars Fact,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [10]:
#Use Pandas to convert the data to a HTML table string
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars Fact</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC

In [11]:
#Visit USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [12]:
#create soup instance to scrape images
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#empty list to store image dictionaries and find image links
hemisphere_image_urls = []

image_links = soup.find_all('div',class_='description')

In [13]:
for link in image_links:
    image_url = {}
    #Find name of Hemisphere link, use to go to click on link
    hemisphere = link.find('h3').text
    image_url['title'] = hemisphere
    browser.click_link_by_partial_text(hemisphere)
    time.sleep(2)
    #reset soup instance, fetch image link on new site
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    img = soup.find('img',class_='wide-image')['src']
    img = 'https://astrogeology.usgs.gov' + img
    image_url['img_url'] = img 
    hemisphere_image_urls.append(image_url)
    #go back to original page and repeat process for other links
    browser.back()
    time.sleep(2)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

In [14]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]